In [1]:
import numpy as np
import pandas as pd
import openturns as ot
import otbenchmark as otb
from matplotlib import pyplot as plt
from bancs import *

In [3]:
omegap = "((kp / mp) ^ 0.5)"
omegas = "((ks / ms) ^ 0.5)"
gamma = "(ms / mp)"
zetaa = "((zetap + zetas) / 2)"
omegaa = f"(({omegap} + {omegas}) / 2)"
theta = f"(({omegap} - {omegas}) / {omegaa})"
Exs2 = f"(pi_ * S0 / (4 * zetas * {omegas} ^ 3) * ({zetaa} * zetas / (zetap * zetas * (4 * {zetaa}^2 + {theta}^2) + {gamma} * {zetaa}^2) * ((zetap * {omegap}^3 + zetas * {omegas}^3) * {omegap} / (4 * {zetaa} * {omegaa}^4))))"
g_string = f"Fs - ks * 3 * {Exs2}^0.5"
g = ot.SymbolicFunction(["mp", "ms", "kp", "ks", "zetap", "zetas", "Fs", "S0"], [g_string])

mp =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(1.5,     0.1))
ms =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.01,    0.1))
kp =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(1.,      0.2))
ks =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.01,    0.2))
zetap = ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.05,    0.4))
zetas = ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(0.02,    0.5))
Fs =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(27.5,    0.1))
S0 =    ot.ParametrizedDistribution(ot.LogNormalMuSigmaOverMu(100,     0.1))

X = ot.ComposedDistribution([mp,ms,kp,ks,zetap,zetas,Fs,S0])
X.setDescription(["$m_p$", "$m_s$", "$k_p$", "$k_s$", "$\\zeta_p$", "$\\zeta_s$", "$F_s$", "$S_0$"])
Y = ot.CompositeRandomVector(g, ot.RandomVector(X))
failure_event = ot.ThresholdEvent(Y, ot.LessOrEqual(), 0.)
pf_ref = 3.78 * 1e-7

In [4]:
N = int(1e4)
m = int(1 + np.floor(N ** (2 / (4 + 8))))
m

5

In [5]:
bancs = BANCS(failure_event, N=N, M=m, p0=0.1, lower_truncatures=[0.] * 8)
quantiles = bancs.run()
pf = bancs.compute_pf()
print("Quantiles =", quantiles)
print("Proba EBC = {:.2e}".format(pf))
print("Relative error = {:.2%}".format(np.abs(pf - pf_ref) / pf_ref))
df = bancs.df.copy()
nb_subset = len(df['Subset'].unique())

Quantiles = [17.20988291 11.58073875  7.32726597  2.97724774 -1.82346416]
Proba EBC = 3.30e-07
Relative error = 12.64%


In [6]:
df

,Subset,$m_p$,$m_s$,$k_p$,$k_s$,$\zeta_p$,$\zeta_s$,$F_s$,$S_0$,IS_weight,Y,Quantile,Failed,gFailed
0,0.0,1.542740,0.009764,1.049404,0.008752,0.044248,0.025293,24.327360,104.549255,1.000000e+00,18.836620,17.209883,0.0,0
1,0.0,1.680526,0.008945,0.797075,0.008875,0.052762,0.012549,25.029429,103.255503,1.000000e+00,22.560100,17.209883,0.0,0
2,0.0,1.337209,0.009897,1.339011,0.012631,0.030667,0.015794,26.896467,81.078196,1.000000e+00,19.528999,17.209883,0.0,0
3,0.0,1.241632,0.009319,1.058307,0.010183,0.057907,0.011923,28.749714,95.109668,1.000000e+00,22.555232,17.209883,0.0,0
4,0.0,1.435136,0.010485,1.085432,0.009020,0.045589,0.013160,25.219789,86.067678,1.000000e+00,16.438035,17.209883,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4.0,1.606040,0.010929,1.209193,0.006829,0.018843,0.010881,18.155101,120.899980,3.909181e-06,4.401076,-1.823464,0.0,0
9996,4.0,1.437888,0.009201,1.659486,0.011847,0.017696,0.004653,24.554994,99.769170,1.679507e-03,6.569014,-1.823464,0.0,0
9997,4.0,1.610024,0.011732,0.843055,0.008278,0.022951,0.003676,20.046814,102.720555,4.904275e-05,8.331023,-1.823464,0.0,0
9998,4.0,1.425978,0.011090,1.229960,0.010445,0.022606,0.002441,18.432240,107.431170,4.260575e-08,-1.233904,-1.823464,1.0,1


In [7]:
multindex = pd.MultiIndex.from_product([["Subset {}".format(l + 1) for l in range(nb_subset)], list(X.getDescription())], names=["sample", "variable"])
df_tsa = pd.DataFrame([], columns=multindex, index=["HSIC", "R2HSIC", "pvalue_asymptotic", "pvalue_permutation"])
df_csa = pd.DataFrame([], columns=multindex, index=["HSIC", "R2HSIC", "pvalue_permutation"])

In [8]:
f = ot.SymbolicFunction(["x", "s"], ["exp(-x/s)"])
estimatorType = ot.HSICUStat()
for l in range(nb_subset):
    ssample = df[(df["Subset"]==l)][list(X.getDescription())].values
    ssample_output = df[(df["Subset"]==l)]["Y"].values.reshape(-1, 1)
    ssample_stds = ssample.std(axis=0)
    ssample_stds = np.append(ssample_stds, ssample_output.std())
    #ssample_stds.append(ssample_output.computeStandardDeviation()[0])
    kernel_collection = [ot.SquaredExponential([std_i]) for std_i in ssample_stds]
    dist2criticalDomain = ot.DistanceToDomainFunction(ot.Interval(quantiles[l], float("inf")))
    s = 0.1 * ssample_output.std()
    phi = ot.ParametricFunction(f, [1], [s])
    filterFunction = ot.ComposedFunction(phi, dist2criticalDomain)
    # TSA with HSIC indices
    tsa = ot.HSICEstimatorTargetSensitivity(kernel_collection, ot.Sample(ssample), ot.Sample(ssample_output), 
                                            ot.HSICUStat(), filterFunction)
    df_tsa.loc["HSIC", "Subset {}".format(l + 1)] = list(tsa.getHSICIndices())
    df_tsa.loc["R2HSIC", "Subset {}".format(l + 1)] = list(tsa.getR2HSICIndices())
    df_tsa.loc["pvalue_asymptotic", "Subset {}".format(l + 1)] = list(tsa.getPValuesAsymptotic())
    #df_tsa.loc["pvalue_permutation", "Subset {}".format(l + 1)] = list(tsa.getPValuesPermutation())
    #CSA with HSIC indices
    csa = ot.HSICEstimatorConditionalSensitivity(kernel_collection, ot.Sample(ssample), ot.Sample(ssample_output), filterFunction)
    df_csa.loc["HSIC", "Subset {}".format(l + 1)] = list(csa.getHSICIndices())
    df_csa.loc["R2HSIC", "Subset {}".format(l + 1)] = list(csa.getR2HSICIndices())
    #df_csa.loc["pvalue_permutation", "Subset {}".format(l + 1)] = list(csa.getPValuesPermutation())

In [ ]:
df_tsa

sample              Subset 1                                          \
variable                  X0        X1        X2        X3        X4   
HSIC                0.000004      -0.0  0.000046  0.000021  0.000019   
R2HSIC              0.003985 -0.000305  0.042949  0.019455   0.02052   
pvalue_asymptotic   0.113149  0.414478  0.000005  0.000027  0.000637   
pvalue_permutation  0.089109  0.366337       0.0       0.0       0.0   

sample                                            Subset 2            ...  \
variable                  X5        X6        X7        X0        X1  ...   
HSIC                 0.00001  0.000049 -0.000003  0.000008 -0.000001  ...   
R2HSIC              0.010231  0.046721 -0.002937  0.005387 -0.000532  ...   
pvalue_asymptotic   0.007518  0.000004  0.969864  0.066522  0.434154  ...   
pvalue_permutation       0.0       0.0  0.980198  0.049505  0.524752  ...   

sample              Subset 4            Subset 5                      \
variable                  X6        X7        X0        X1        X2   
HSIC                0.000005  0.000014       0.0 -0.000001       0.0   
R2HSIC              0.006323  0.013661  0.000764 -0.002333  0.000219   
pvalue_asymptotic   0.035487  0.002163  0.288711  0.832906  0.335918   
pvalue_permutation  0.069307       0.0  0.336634  0.831683  0.366337   

sample                                                                
variable                  X3        X4        X5        X6        X7  
HSIC                    -0.0  0.000038  0.000023      -0.0  0.000001  
R2HSIC             -0.000829  0.085536  0.054621 -0.001013   0.00274  
pvalue_asymptotic   0.518882       0.0  0.000004  0.535895   0.11823  
pvalue_permutation  0.435644       0.0       0.0  0.544554  0.128713  

[4 rows x 40 columns]

In [ ]:
df_csa

sample              Subset 1                                          \
variable                  X0        X1        X2        X3        X4   
HSIC                0.000443  0.001567  0.004737  0.005491  0.000591   
R2HSIC              0.006585  0.022305  0.069151  0.077858  0.009777   
pvalue_permutation  0.267327       0.0       0.0       0.0  0.039604   

sample                                            Subset 2            ...  \
variable                  X5        X6        X7        X0        X1  ...   
HSIC                0.000301  0.025521  0.000137  0.002575  0.000939  ...   
R2HSIC              0.004862   0.38115  0.001972  0.034648  0.013195  ...   
pvalue_permutation  0.356436       0.0  0.891089       0.0  0.039604  ...   

sample              Subset 4            Subset 5                      \
variable                  X6        X7        X0        X1        X2   
HSIC                0.006169  0.002506  0.000158  0.000769  0.000236   
R2HSIC              0.091901  0.030626  0.002533  0.011977  0.004153   
pvalue_permutation       0.0       0.0  0.722772  0.029703  0.366337   

sample                                                                
variable                  X3        X4        X5        X6        X7  
HSIC                0.000625   0.00241  0.004133  0.005118  0.001033  
R2HSIC              0.009987  0.036672  0.068898  0.074347  0.015941  
pvalue_permutation  0.039604       0.0       0.0       0.0  0.009901  

[3 rows x 40 columns]

In [63]:
#df_tsa_plot = df_tsa.T.reorder_levels(["variable", "sample"])
#df_csa_plot = df_csa.T.reorder_levels(["variable", "sample"])
#df_tsa_plot.to_csv("rosa_results/oscillator_tsa.csv")
#df_csa_plot.to_csv("rosa_results/oscillator_csa.csv")
#df_tsa_plot